In [3]:
%matplotlib tk

import numpy as np
import matplotlib.pyplot as plt

FILE = 'data/pills-s0.npy'

X = np.load(FILE)

print(X.shape)

fig, ax = plt.subplots()
ax.plot(X[:,:-3], alpha=.7)
ax.set_ylabel('gyroscope')
ax.set_xlabel('time (milliseconds)')
ax.set_ylim(-5000, 5000)
plt.show()

(4506, 6)
